In [3]:


# %%python
from datetime import datetime, timedelta
from sqlmodel import SQLModel, create_engine, Session,select

# your model import (adjust the path as needed)
from fateforger.agents.schedular.models.calendar_event import (
    CalendarEvent,
    EventDateTime,
    CreatorOrganizer,
)

# 1) create an in-memory SQLite DB and reflect models
engine = create_engine("sqlite:///:memory:")
SQLModel.metadata.create_all(engine)

# 2) open a session, add a sample event, commit & refresh
with Session(engine) as session:
    ev = CalendarEvent(

        summary="🧪 Test Meeting",
        description="This is just a test insert",
        location="Virtual",
        creator=CreatorOrganizer(email="tester@example.com", display_name="Tester"),
        start=EventDateTime(date_time=datetime.utcnow(), time_zone="UTC"),
        end=EventDateTime(date_time=datetime.utcnow() + timedelta(hours=1), time_zone="UTC"),
    )
    session.add(ev)
    session.commit()
    session.refresh(ev)
    print("Inserted:", ev)

# 3) query all events and print
with Session(engine) as session:
    statement = select(CalendarEvent)
    all_events = session.exec(statement).all()
    print("All events in DB:", all_events)

Inserted: id=1 status=None description='This is just a test insert' color_id=None start=EventDateTime(date=None, date_time=datetime.datetime(2025, 7, 28, 16, 38, 6, 158768), time_zone='UTC') source=None extended_properties=None event_type=None calendar_id=None google_event_id=None summary='🧪 Test Meeting' location='Virtual' creator=CreatorOrganizer(id=None, email='tester@example.com', display_name='Tester', self_=None) end=EventDateTime(date=None, date_time=datetime.datetime(2025, 7, 28, 17, 38, 6, 158919), time_zone='UTC') transparency=None reminders=None
All events in DB: [CalendarEvent(id=1, status=None, description='This is just a test insert', color_id=None, start=EventDateTime(date=None, date_time=datetime.datetime(2025, 7, 28, 16, 38, 6, 158768), time_zone='UTC'), source=None, extended_properties=None, event_type=None, calendar_id=None, google_event_id=None, summary='🧪 Test Meeting', location='Virtual', creator=CreatorOrganizer(id=None, email='tester@example.com', display_name='